# Import


In [26]:
import aerosandbox as asb
import aerosandbox.numpy as np
import neuralfoil as nf
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
import math

# Step 1: Take off mass

In [27]:
g = 9.81 

eng_q = 2 
con_cell_num = 6 # количество параллельных ячеек в батарее
cell_U = 4.2 # Напряжение каждой ячейки
prop_D = 22 # Диаметрр пропеллера в дюймах

LD_ratio_cruise = 14 # Отношение подъемной силы к силы лобового сопротивления
prop_efficiency = 0.75 # КПД винта
battery_specific_energy = 216 
payload_mass = 1.5 # Масса полезной нагрузки
flight_time = 90 # Время полета в минутах
flight_time_hr = flight_time / 60 # Время полета в часах
cruise_speed = 90 # Скорость полета в км/ч
cruise_speed_ms = cruise_speed / 3.6 # Скорость полета в км/ч
takeoff_mass_reserve = 1.07 # Коэффициент запаса массы для взлета
battery_mass_reserve = 1.1 # Коэффициент запаса массы аккумулятора
k = 0.3 #
n = 2.3 #
prop_fasteners = 0.015 # коэффициент добавочной массы креплений в СУ
e = 0.78 # Коэффициент Освальда


## Zero approximation of takeoff mass

In [28]:
zero_apr_mass = 5.147 * (payload_mass ** 0.849)
zero_apr_weight = zero_apr_mass * g
print(zero_apr_mass)
print(zero_apr_weight)

7.26199001085213
71.2401220064594


## First approximation of takeoff mass

In [29]:
equipment_mass = 0.46
stat_structure_mass = zero_apr_mass * 0.3
req_power = zero_apr_weight * cruise_speed / LD_ratio_cruise / prop_efficiency
one_eng_power = req_power / eng_q
eng_mass = 0.889 * (one_eng_power ** -0.288) * ((con_cell_num * cell_U) **  0.1588)
prop_mass = k * (prop_D ** n) / 1000
power_mass = eng_mass + prop_mass + prop_fasteners
battery_mass = (req_power * flight_time_hr / battery_specific_energy) * battery_mass_reserve
m0_new1 = (payload_mass + stat_structure_mass + equipment_mass + power_mass + battery_mass) * takeoff_mass_reserve

print("First approximation takeoff mass (kg):", m0_new1)
print("Power required (W):", req_power)
print("Motor mass (kg):", eng_mass)
print("Propeller mass (kg):", prop_mass)
print("Power plant mass (kg):", power_mass)
print("Battery mass (kg):", battery_mass)
print("Structure mass (kg):", stat_structure_mass)

mass_diff = (zero_apr_mass - m0_new1) / (zero_apr_mass * 100)
print("Mass difference (%):", mass_diff)

First approximation takeoff mass (kg): 10.133759849953663
Power required (W): 610.6296171982234
Motor mass (kg): 0.28565167860570095
Propeller mass (kg): 0.36702311818421196
Power plant mass (kg): 0.6676747967899129
Battery mass (kg): 4.664531798041984
Structure mass (kg): 2.1785970032556388
Mass difference (%): -0.003954521880104536


## Second approximation of takeoff mass

In [30]:
zero_apr_mass = m0_new1
req_power = zero_apr_weight * cruise_speed / LD_ratio_cruise / prop_efficiency
one_eng_power = req_power / eng_q
eng_mass = 0.889 * (one_eng_power ** -0.288) * ((con_cell_num * cell_U) **  0.1588)
prop_mass = k * (prop_D ** n) / 1000
power_mass = eng_mass + prop_mass + prop_fasteners
battery_mass = (req_power * flight_time_hr / battery_specific_energy) * battery_mass_reserve
stat_structure_mass = zero_apr_mass * 0.3
m0_new2 = (payload_mass + stat_structure_mass + equipment_mass + power_mass + battery_mass) * takeoff_mass_reserve

print("\nSecond approximation takeoff mass (kg):", m0_new2)
print("Power required (W):", req_power)
print("Motor mass (kg):", eng_mass)
print("Propeller mass (kg):", prop_mass)
print("Power plant mass (kg):", power_mass)
print("Battery mass (kg):", battery_mass)
print("Structure mass (kg):", stat_structure_mass)

mass_diff = (zero_apr_mass - m0_new1) / (zero_apr_mass * 100)
print("Mass difference (%):", mass_diff)


Second approximation takeoff mass (kg): 11.055597968305257
Power required (W): 610.6296171982234
Motor mass (kg): 0.28565167860570095
Propeller mass (kg): 0.36702311818421196
Power plant mass (kg): 0.6676747967899129
Battery mass (kg): 4.664531798041984
Structure mass (kg): 3.0401279549860987
Mass difference (%): 0.0


# Step 2: Определение параметров завязки
## Input value 1

In [31]:
stall_speed = 65 # скорость сваливания (км/ч)
air_density = 1.225 # плотность воздуха на взлете
CLmax = 1.4 # коэффициент подъемной силы крыла максимальный (по статистике)
CLmin = -0.8 # коэффициент подъемной силы крыла минимальный (по поляре профиля)
CL = 5.25 # производная CL (по поляре профиле)
WP = 0.1 # энерговооруженность (по прототипу)



## Calculation 1

In [32]:
stall_speed_ms = stall_speed / 3.6 # скорость сваливания (м/с)
dynamic_pressuare = 0.5 * air_density * stall_speed_ms ** 2 # динамическое давление при сваливании 
WS = CLmax * dynamic_pressuare # Потребная нагрузка на крыло при сваливании

print("Динамическое давление при сваливании:", dynamic_pressuare)
print("Потребная нагрузка на крыло при сваливании:", WS)

Динамическое давление при сваливании: 199.67689043209873
Потребная нагрузка на крыло при сваливании: 279.5476466049382


## Calculation 2

In [33]:
take_off_weight = m0_new2 * g
S_wing = WS / take_off_weight

print("Wing area (S):", S_wing)

Wing area (S): 2.5775351451964013


## Сalculation of battery parameters

In [38]:
series_cells_n = 6
parallel_cells_n = 4 
battery_weight = battery_mass * g
cell_U_min = 3
cell_U_max = 4.2
cell_capacity_nom_A = 4.2 
cell_capacity_nom_W = 15.5
max_current_cell = 45
cell_mass = 0.07
battery_mass_coef = 1.12

screws_n = 2
blades_n = 2
Kp = 0.0995

cell_total = series_cells_n * parallel_cells_n
pack_cell_U_min = series_cells_n * cell_U_min
pack_cell_U_max = series_cells_n * cell_U_max
pack_cell_U = series_cells_n * cell_U
pack_cell_capacity_nom_A = parallel_cells_n * cell_capacity_nom_A
pack_cell_capacity_nom_W = parallel_cells_n * cell_capacity_nom_W
pack_max_current_cell = parallel_cells_n * max_current_cell
pack_cell_mass = cell_total * cell_mass * battery_mass_coef
battery_pack_q = 1
total_battery_mass = pack_cell_mass * battery_pack_q
total_battery_capacity = battery_pack_q * pack_cell_capacity_nom_A

max_cell_q = battery_weight / battery_mass_coef / cell_mass

print("Емкость АБ:", pack_cell_capacity_nom_A, "А*ч")
print("Масса АБ:", total_battery_mass)
print("Емкость всех АБ при параллельном подключении:", total_battery_capacity, "А*ч")

Емкость АБ: 16.8 А*ч
Масса АБ: 1.8816000000000004
Емкость всех АБ при параллельном подключении: 16.8 А*ч


## Input value 3

In [35]:
SwetS = 3.8
fric_coef = 0.0053
AR = 8
air_density_cruise = 1.1117
dynamic_viscosity_cruise = 0.00001787
Re_cruise = 511979
prop_efficiency_cruise = 0.75
battery_depth = 0.3

## Calculation 3

In [40]:
Cd0 = SwetS * fric_coef
K = 0.3183 / AR / 0.78
wing_loading = WS * 1
dynamic_pressuare_cruise = 0.5 * air_density * (cruise_speed_ms ** 2)
aerodynamic_quality = 1 / ((dynamic_pressuare_cruise*(Cd0 / wing_loading)) * (wing_loading * (K / dynamic_pressuare_cruise)))

eng_efficiency = prop_efficiency * 0.95
MbatM0 = battery_weight / (m0_new2 * g)
payload_weight = payload_mass * g

print("Распологаемая нагрузка на крыло на крейсерском режиме:", wing_loading)
print("Отношение массы батареи к массе всего БЛА в течении заданного времени:", MbatM0)

Распологаемая нагрузка на крыло на крейсерском режиме: 279.5476466049382
Отношение массы батареи к массе всего БЛА в течении заданного времени: 0.4219158304611381


## Input value 4

In [ ]:
wing_narrowing = 0.57
fuselage_lenght = 
fuselage_width =

## Calculation 4

In [ ]:
wingspan = (AR * S_wing) ** 0.5
root_chord_lenght = (2 * wing_narrowing) / (wingspan * (1 + wing_narrowing))
tip_chord_lenght = wingspan * root_chord_lenght
aerodynamic_chord_lenght = 0.6667 * root_chord_lenght * (1 + wing_narrowing + wing_narrowing ** 2) / (1 + wing_narrowing)